In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import openml
import seaborn as sn

In [2]:
def obter_dataset(id):


    dataset = openml.datasets.get_dataset(id, download_data=True, download_qualities=True, download_features_meta_data=True)
    
    X, y, _, attrs = dataset.get_data(dataset_format="array", target=dataset.default_target_attribute)
    
    df = pd.DataFrame(X, columns=attrs)
    df['target'] = y
    # converter:
    #   0 -> -1
    #   1 -> 1
    df['target'] = 2*y-1
    # erase rows with NaN values
    df = df.dropna(how='any', axis=0)

    csv_path = f'{id}.csv'
    df.to_csv(csv_path, index= False)
    
    return df

In [3]:
list_num_dataset_id = [1464, 1067, 1467, 1504,1494, 1068]
dataframes = {}
for dataset_id in list_num_dataset_id:
    df = obter_dataset(dataset_id)
    dataframes[dataset_id] = df

C:\Users\ricar\AppData\Local\Temp\ipykernel_980\912023531.py:6: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, y, _, attrs = dataset.get_data(dataset_format="array", target=dataset.default_target_attribute)
C:\Users\ricar\AppData\Local\Temp\ipykernel_980\912023531.py:6: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, y, _, attrs = dataset.get_data(dataset_format="array", target=dataset.default_target_attribute)
C:\Users\ricar\AppData\Local\Temp\ipykernel_980\912023531.py:6: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure yo

In [4]:
import numpy as np
from sklearn.model_selection import KFold



def run_cv(X,y,algs,nfolds=10, means_only=False):
    results = {}
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=1111)
    for algo_name, algo in algs:
        results[algo_name] = []
        for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            algo.fit(X_train, y_train, M=40)
            y_pred = algo.predict(X_test)
            results[algo_name].append(accuracy_score(y_test, y_pred))
    results_df = pd.DataFrame.from_dict(results)
    if not means_only:
        return results_df
    else:
        results_means = {}
        for algo_name, algo in algs:
            results_means[algo_name] = [np.mean(results[algo_name])]
        return pd.DataFrame.from_dict(results_means)

In [5]:
import matplotlib.pyplot as plt

def plot_cv(results_cv,metric='Accuracy', title="Cross-validation results for multiple algorithms in a single task"):
    fig, ax = plt.subplots()
    ax.boxplot(results_cv)
    ax.set_xticklabels(results_cv.columns)
    ax.set_ylabel(metric)
    ax.set_title(title)
    plt.show()

In [7]:
from algorithms.adaboost import AdaBoost
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
models = [
    ("AdaBoost Default", AdaBoost()),
    ("AdaBoost KNN", AdaBoost(knn=True))
]

for id in list_num_dataset_id:
    print(f"Dataset {id}:")
    X = dataframes[id].drop(columns=['target'], axis=1)
    y = dataframes[id]['target']
    # Dividindo os dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    for model_name, model in models:
        model.fit(X_train, y_train)
        # Testando o modelo
        y_pred = model.predict(X_test)
    
        # Calculando a acurácia
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Acurácia do modelo para dataset {model_name}: {accuracy}")
    
    print("\n")

Dataset 1464:
Acurácia do modelo para dataset AdaBoost Default: 0.7688888888888888
Acurácia do modelo para dataset AdaBoost KNN: 0.6622222222222223


Dataset 1067:


AssertionError: 